In [1]:
import os
import pandas as pd
from ipywidgets import IntProgress
from IPython.display import display

ModuleNotFoundError: No module named 'ipywidgets'

In [2]:
data_dir = './'

## Processing data files for creating input data

- Loading `.features` files into data frames and concatenating them

In [4]:
instance_ids = []
for basedir, _, files in os.walk(data_dir):
  for file in files:
    if file.endswith('.features'):
        instance_id = os.path.relpath(os.path.join(basedir, file), data_dir)
        instance_ids.append(instance_id)

In [5]:
f = IntProgress(min=0, max=len(instance_ids))
display(f)
xs = []    
for i in range(len(instance_ids)):
  file = instance_ids[i]
  n_series = pd.Series([file[:-9]], name='instance_id')
  data = pd.read_csv(os.path.join(data_dir, file))
  x = pd.merge(n_series, data, left_index=True, right_index=True)
  xs.append(x)
  f.value += 1

IntProgress(value=0, max=775)

In [19]:
x = pd.concat(xs)
len(x)

775

In [7]:
x.head()

,instance_id,nvarsOrig,nclausesOrig,nvars,nclauses,reducedVars,reducedClauses,Pre-featuretime,vars-clauses-ratio,POSNEG-RATIO-CLAUSE-mean,...,CG-coeff-variation,CG-min,CG-max,CG-entropy,cluster-coeff-mean,cluster-coeff-coeff-variation,cluster-coeff-min,cluster-coeff-max,cluster-coeff-entropy,CG-featuretime
0,SAT_Competition2007/industrial/industrial/grie...,729.0,96849.0,729.0,96849.0,0.0,0.0,0.08,0.007527,0.594369,...,-0.0,-1.0,0.0,-512.0,-1.0,-0.0,-1.0,0.0,-0.0,20.60
0,SAT_Competition2007/industrial/industrial/grie...,676.0,86424.0,676.0,86424.0,0.0,0.0,0.06,0.007822,0.594164,...,-0.0,-1.0,0.0,-512.0,-1.0,-0.0,-1.0,0.0,-0.0,20.18
0,SAT_Competition2007/industrial/industrial/grie...,841.0,120147.0,841.0,120147.0,0.0,0.0,0.10,0.007000,0.594738,...,-0.0,-1.0,0.0,-512.0,-1.0,-0.0,-1.0,0.0,-0.0,20.16
0,SAT_Competition2007/industrial/industrial/grie...,1089.0,177375.0,1089.0,177375.0,0.0,0.0,0.16,0.006140,0.595349,...,-0.0,-1.0,0.0,-512.0,-1.0,-0.0,-1.0,0.0,-0.0,20.22
0,SAT_Competition2007/industrial/industrial/grie...,1156.0,194072.0,1156.0,194072.0,0.0,0.0,0.16,0.005957,0.595480,...,-0.0,-1.0,0.0,-512.0,-1.0,-0.0,-1.0,0.0,-0.0,21.04


In [8]:
x = x.sort_values(by=['instance_id'])

## Processing csv files for creating output data

- Loading raw `csv` data
- Creating appropriate data frames for each instance
- Filtering outputs to match inputs

In [11]:
data = pd.concat([pd.read_csv(os.path.join(data_dir, 'SAT12-HAND.csv')), pd.read_csv(os.path.join(data_dir, 'SAT12-INDU.csv'))])

In [12]:
data.head()

,instance_id,repetition,solver name,runtime,runstatus
0,SAT_Competition2011/SAT03/handmade/anton/UNSAT...,1,ebglucose,1200.0,timeout
1,SAT_Competition2011/SAT03/handmade/anton/UNSAT...,1,ebminisat,1200.0,timeout
2,SAT_Competition2011/SAT03/handmade/anton/UNSAT...,1,glucose2,1200.0,timeout
3,SAT_Competition2011/SAT03/handmade/anton/UNSAT...,1,glueminisat,1200.0,timeout
4,SAT_Competition2011/SAT03/handmade/anton/UNSAT...,1,lingeling,1200.0,timeout


In [13]:
instance_ids = []
instance_id = None
n = len(data)
new_row = None
columns = None
ys = []
for i in range(n):
  old_row = data.iloc[i]
  inst_id = old_row['instance_id']
  if inst_id != instance_id:
    if new_row is not None:
      if inst_id in instance_ids:
        continue
      y = pd.DataFrame([new_row], columns=columns)
      ys.append(y)
      instance_ids.append(instance_id)

    new_row = [inst_id]
    columns = ['instance_id']
    instance_id = inst_id
  new_row.append(old_row['runtime'])
  columns.append(old_row['solver name'])

In [14]:
y = pd.concat(ys)
len(y)

1933

In [15]:
y.head()

,instance_id,ebglucose,ebminisat,glucose2,glueminisat,lingeling,lrglshr,minisatpsm,mphaseSAT64,precosat,...,mxc09,gnoveltyp2,sattime,sattimep,clasp2,clasp1,picosat,mphaseSAT,sapperlot,sol
0,SAT_Competition2011/SAT03/handmade/anton/UNSAT...,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,...,1200.00,1200.0,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00
0,SAT_Competition2009/CRAFTED/Hard/contest05/phn...,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,...,1200.00,1200.0,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00
0,SAT_Competition2009/CRAFTED/edgematching/all/e...,71.69,264.99,45.89,174.07,1010.96,594.68,719.54,76.87,36.00,...,163.60,1200.0,117.42,998.09,7.11,16.80,0.02,27.79,67.31,155.62
0,SAT_Competition2009/CRAFTED/Difficult/contest0...,100.92,75.20,67.68,23.11,151.40,99.89,746.98,57.28,24.15,...,82.36,1200.0,1200.00,1200.00,26.14,20.14,970.90,56.37,311.85,153.90
0,SAT_Competition2011/SAT03/handmade/anton/UNSAT...,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,...,1200.00,1200.0,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00,1200.00


In [16]:
def filter_by_available_instances(df):
  filter_array = []
  available_instances = list(x['instance_id'])
  for i in range(len(df)):
    value = df.iloc[i]['instance_id'] in available_instances
    filter_array.append(value)
  return filter_array

In [17]:
all_y_instances = len(y)
y = y.loc[filter_by_available_instances, :]
filtered_y_instances = len(y)
y = y.sort_values(by=['instance_id'])
print('Found {}/{} instances'.format(filtered_y_instances, all_y_instances))

Found 775/1933 instances


## Final checks

- The size of X and Y must be the same

In [22]:
assert(len(x) == len(y))

## Saving the data

In [9]:
x.to_csv(os.path.join(data_dir, 'chosen_data', 'data_x.csv'), index=False)

In [18]:
y.to_csv(os.path.join(data_dir, 'chosen_data', 'data_y.csv'), index=False)